
# TCP/IP Encapsulation Project – Student Guide (English)

**Goal:** Craft IPv4+TCP packets. On Linux/macOS, use raw sockets; on Windows, automatically fall back to **Scapy + Npcap**. <br>
**Flow:** CSV (Application Messages) → Notebook (Encapsulation Simulation) → Wireshark (Capture) → Report (Explanation) <br>
**Safety:** Educational use only. Prefer loopback/VM. Admin/root privileges usually required.


## Prerequisites
- **Linux/macOS**: `sudo jupyter lab` → capture `lo` in Wireshark → run `demo_send()`
- **Windows**:
  1) Install Wireshark/Npcap with *WinPcap API-compatible mode* + *Support loopback traffic*.
  2) `pip install scapy pandas`
  3) Run Jupyter **as Administrator**
  4) Capture **Npcap Loopback Adapter** in Wireshark
  5) Run `demo_send(iface='Npcap Loopback Adapter')`


In [1]:
pip install scapy pandas

Note: you may need to restart the kernel to use updated packages.



## Step 1 — Load Your CSV (Input)
1. Place your CSV file (e.g., `group05_http_input.csv`) in the same folder as this notebook.
2. The CSV must contain the following columns: `msg_id, app_protocol, src_app, dst_app, message, timestamp`.
3. In the next cell, set `CSV_PATH` to your file name and run the cell.
4. Verify that the preview shows your rows correctly.


In [1]:
#TODO: Load CSV file with messages into pandas DataFrame, 

# Replace 'path_to_your_file.csv' with the actual file path
import pandas as pd
filename =  "C:/פרויקט רשתות/group02_http_input1.csv" # "path_to_your_file.csv"
messages_df = pd.read_csv(filename)



## Step 2 — Validate the Schema
The notebook will automatically check the CSV header. If a required column is missing, you will see an error message.
- If validation fails, fix your CSV and re-run Step 1.
- If it passes, continue to Step 3.


In [2]:
# TODO: Run the cell to validate the CSV file format

def validate_csv_format(df: pd.DataFrame):
    expected_columns = ["app_protocol", "src_port", "dst_port", "message", "timestamp"]
    for col in expected_columns:
        if col not in df.columns:
            raise ValueError(f"Missing expected column: {col}")
        
messages_df['message'] = messages_df['message'].fillna('')  # Fill NaN messages with empty strings
validate_csv_format(messages_df)
print(messages_df)

    msg-id  timestamp   src_port   dst_port app_protocol  Length  \
0      194   3.173704  127.0.0.1  127.0.0.1          TCP      56   
1      195   3.173840  127.0.0.1  127.0.0.1          TCP      56   
2      196   3.173884  127.0.0.1  127.0.0.1          TCP      44   
3      197   3.174309  127.0.0.1  127.0.0.1          TCP      57   
4      199   3.174379  127.0.0.1  127.0.0.1          TCP      44   
5      201   3.174507  127.0.0.1  127.0.0.1          TCP      60   
6      202   3.174534  127.0.0.1  127.0.0.1          TCP      44   
7      339   4.177726  127.0.0.1  127.0.0.1          TCP      57   
8      340   4.177868  127.0.0.1  127.0.0.1          TCP      44   
9      341   4.178064  127.0.0.1  127.0.0.1          TCP      60   
10     342   4.178080  127.0.0.1  127.0.0.1          TCP      44   
11     382   5.178783  127.0.0.1  127.0.0.1          TCP      57   
12     383   5.178852  127.0.0.1  127.0.0.1          TCP      44   
13     386   5.178949  127.0.0.1  127.0.0.1     


## Step 3 — Map to TCP/IP Layers (Encapsulation)
This notebook will map each application message to the TCP/IP stack:
- Application → Transport (e.g., TCP/UDP headers)
- Internet (IP headers)
- Link (frame headers/footers)

Just run the cell(s) in this section to see the derived structures.



In [3]:
# NOTE: Follow the step-by-step markdown cells above. Set paths only where indicated.
import socket, struct, random, time, platform
from typing import Optional

IS_WINDOWS = (platform.system() == 'Windows')
try:
    from scapy.all import IP as SCAPY_IP, TCP as SCAPY_TCP, Raw as SCAPY_Raw, send as scapy_send, get_if_list
    HAVE_SCAPY = True
except Exception as e:
    HAVE_SCAPY = False
    SCAPY_IMPORT_ERR = e
IS_WINDOWS, HAVE_SCAPY

(True, True)

In [4]:
# Function to calculate checksum
def checksum(data: bytes) -> int:
    if len(data) % 2:
        data += b'\0'
    res = sum(struct.unpack('!%dH' % (len(data)//2), data))
    while res >> 16:
        res = (res & 0xFFFF) + (res >> 16)
    return ~res & 0xFFFF

# Helper function to display the data 
def hexdump(data: bytes, width: int=16):
    for i in range(0, len(data), width):
        chunk = data[i:i+width]
        hex_bytes = ' '.join(f'{b:02x}' for b in chunk)
        ascii_bytes = ''.join(chr(b) if 32 <= b < 127 else '.' for b in chunk)
        print(f"{i:04x}  {hex_bytes:<{width*3}}  {ascii_bytes}")


In [5]:
def build_ip_header(src_ip: str, dst_ip: str, payload_len: int, proto: int=socket.IPPROTO_TCP) -> bytes:
    version_ihl = (4 << 4) + 5
    tos = 0
    total_length = 20 + payload_len
    identification = random.randint(0, 65535)
    flags_fragment = 0
    ttl = 64
    header_checksum = 0
    src = socket.inet_aton(src_ip)
    dst = socket.inet_aton(dst_ip)
    ip_header = struct.pack('!BBHHHBBH4s4s',
                             version_ihl, tos, total_length, identification,
                             flags_fragment, ttl, proto, header_checksum,
                             src, dst)
    chksum = checksum(ip_header)
    ip_header = struct.pack('!BBHHHBBH4s4s',
                             version_ihl, tos, total_length, identification,
                             flags_fragment, ttl, proto, chksum,
                             src, dst)
    return ip_header


In [6]:
def build_tcp_header(src_ip: str, dst_ip: str, src_port: int, dst_port: int, payload: bytes=b'',
                     seq: Optional[int]=None, ack_seq: int=0, flags: int=0x02, window: int=65535) -> bytes:
    if seq is None:
        seq = random.randint(0, 0xFFFFFFFF)
    doff_reserved = (5 << 4)
    checksum_tcp = 0
    urg_ptr = 0
    tcp_header = struct.pack('!HHLLBBHHH',
                              src_port, dst_port, seq, ack_seq,
                              doff_reserved, flags, window,
                              checksum_tcp, urg_ptr)
    placeholder = 0
    protocol = socket.IPPROTO_TCP
    tcp_length = len(tcp_header) + len(payload)
    pseudo_header = struct.pack('!4s4sBBH',
                                socket.inet_aton(src_ip), socket.inet_aton(dst_ip),
                                placeholder, protocol, tcp_length)
    chksum = checksum(pseudo_header + tcp_header + payload)
    tcp_header = struct.pack('!HHLLBBH H H',
                              src_port, dst_port, seq, ack_seq,
                              doff_reserved, flags, window,
                              chksum, urg_ptr)
    return tcp_header


### Cross‑Platform Transport
- Linux/macOS: raw sockets (we include the IP header)
- Windows: Scapy + Npcap fallback (raw TCP sockets are blocked by the OS)

In [7]:
class RawTcpTransport:
    def __init__(self, src_ip: str, dst_ip: str, src_port: int, dst_port: int, iface: Optional[str]=None):
        self.src_ip = src_ip
        self.dst_ip = dst_ip
        self.src_port = src_port
        self.dst_port = dst_port
        self.iface = iface
        self.windows_fallback = IS_WINDOWS
        if not self.windows_fallback:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_RAW)
        else:
            if not HAVE_SCAPY:
                raise RuntimeError(
                    f"Windows detected but Scapy is not available: {SCAPY_IMPORT_ERR}.\n"
                    "Install with: pip install scapy. Ensure Npcap is installed with loopback support."
                )

    def encapsulate(self, data: bytes, flags: int=0x02) -> bytes:
        tcp = build_tcp_header(self.src_ip, self.dst_ip, self.src_port, self.dst_port, data, flags=flags)
        ip  = build_ip_header(self.src_ip, self.dst_ip, len(tcp) + len(data))
        return ip + tcp + data

    def send(self, data: bytes, flags: int=0x02):
        if not self.windows_fallback:
            pkt = self.encapsulate(data, flags=flags)
            self.sock.sendto(pkt, (self.dst_ip, 0))
        else:
            scapy_pkt = SCAPY_IP(src=self.src_ip, dst=self.dst_ip)/SCAPY_TCP(sport=self.src_port, dport=self.dst_port, flags=flags)/SCAPY_Raw(data)
            chosen_iface = self.iface
            if chosen_iface is None and self.dst_ip in ("127.0.0.1", "::1"):
                chosen_iface = "Npcap Loopback Adapter"
            scapy_send(scapy_pkt, verbose=False, iface=chosen_iface)


In [8]:
# find interface name for Windows
if IS_WINDOWS and HAVE_SCAPY:
    try:
        print('\n'.join(get_if_list()))
    except Exception as e:
        print('Could not list interfaces:', e)


\Device\NPF_{4089C87E-3EBE-4731-8BC4-6226EE899EB0}
\Device\NPF_{591F9F93-EC69-4FCE-A99E-DA204675A091}
\Device\NPF_{EB6966CD-7232-430E-BBFA-C673DB942207}
\Device\NPF_{98A9F56B-916F-471D-9769-B6CA69680395}
\Device\NPF_{132851DD-4116-4C11-8339-24ABDED1034A}
\Device\NPF_{15697119-DFA2-4E20-9E1F-BA5514325E23}
\Device\NPF_Loopback


In [ ]:
# Preview packet structure
src_ip = '127.0.0.1'
dst_ip = '127.0.0.1'
src_port = random.randint(1024, 65535)
dst_port = 12345
payload = b'Hello Packet (preview)'
pkt_preview = build_ip_header(src_ip, dst_ip, 20 + len(payload)) + build_tcp_header(src_ip, dst_ip, src_port, dst_port, payload) + payload
hexdump(pkt_preview)


0000  45 00 00 3e 16 6a 00 00 40 06 24 3d 7f 00 00 01   E..>.j..@.$=....
0010  c0 a8 00 6a 45 5d d4 31 6d 5d 21 47 00 00 00 00   ...jE].1m]!G....
0020  50 02 ff ff b3 b2 00 00 48 65 6c 6c 6f 20 50 61   P.......Hello Pa
0030  63 6b 65 74 20 28 70 72 65 76 69 65 77 29         cket (preview)



## Step 4 — Capture in Wireshark
1. Start capture in Wireshark.
2. Run the transmit/simulation cells in this notebook.
3. Observe packets appearing in Wireshark (timing may vary by system).
4. Stop the capture and save the file as `.pcap`.

### Suggested Wireshark Filters
- `ip.addr == 127.0.0.1 && tcp.port == 12345`
- `tcp.flags.syn == 1 && tcp.flags.ack == 0 && tcp.port == 12345`
- `tcp.flags.push == 1 && tcp.flags.ack == 1 && tcp.port == 12345`



## Step 5 — Generate/Synthesize Traffic
The notebook will simulate the transmission of your messages. You do not need to change parameters unless instructed in comments.
- Make sure Wireshark is open and ready to capture on your active interface.
- Consider applying a simple filter (e.g., `tcp port 80` for HTTP) to focus the view.


In [ ]:
# Create transport instance
src_ip = '127.0.0.1'
dst_ip = '127.0.0.1'
src_port = random.randint(1024, 65535)
dst_port = 12345
iface = "\\Device\\NPF_Loopback"  # Set to "Npcap Loopback Adapter" on Windows if needed
transport = RawTcpTransport(src_ip, dst_ip, src_port, dst_port, iface=iface)

In [11]:
def demo_send(num_packets: int=3, delay_sec: float=1.0, flags: int=0x02):
    for i in range(num_packets):
        payload = f'Hello Packet {i}'.encode()
        print(f"שולח חבילה {i} עם דגל {hex(flags)}...") # הוסף את השורה הזו
        transport.send(payload, flags=flags)
        print("הפקודה עברה את ה-Transport!")
        time.sleep(delay_sec)


### Run (commented for safety)
- Linux/macOS: `demo_send()`
- Windows loopback: `demo_send(iface='Npcap Loopback Adapter')`
- Try flags: `flags=0x18` (PSH+ACK), `0x10` (ACK), `0x01` (FIN), `0x04` (RST)


In [12]:
demo_send(num_packets=3, delay_sec=1.0, flags=0x02)


שולח חבילה 0 עם דגל 0x2...


c:\Users\itay5\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\scapy\sendrecv.py:479: SyntaxWarning: 'iface' has no effect on L3 I/O send(). For multicast/link-local see https://scapy.readthedocs.io/en/latest/usage.html#multicast
  warnings.warn(


הפקודה עברה את ה-Transport!
שולח חבילה 1 עם דגל 0x2...
הפקודה עברה את ה-Transport!
שולח חבילה 2 עם דגל 0x2...
הפקודה עברה את ה-Transport!


### Send Messages from CSV file 

Iterate over the rows and send message by message

In [13]:
#Send messages from CSV file
for index, row in messages_df.iterrows():
    # Extract message details from the DataFrame row
    message = row['message']
    message = f"test message {index}" if not message else message
    # Send the message using the RawTcpTransport class
    # (You may need to adjust flags and other parameters as needed)
    
    #TODO: uncomment the line below to send the messages
    transport.send(message.encode(), flags=0x18)  # Example with PSH+ACK flags
    print("הפקודה עברה את ה-Transport!")
    time.sleep(0.1)  # Optional delay between messages

הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!
הפקודה עברה את ה-Transport!



## Step 6 — Analyze and Explain
In your **report**:
- Explain how the CSV application messages became packets/frames through encapsulation.
- Use **Wireshark screenshots** to illustrate headers, ports, and payloads.
- Link observations back to your CSV rows (e.g., message IDs).



## Deliverables Checklist
- [ ] CSV input file.
- [ ] Executed notebook (with outputs).
- [ ] Wireshark .pcap capture.
